# Import Data

In [ ]:
import sql_functions
import get_user_activity_data
import plotly.express as px
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# manually getting strava data
access_token = get_user_activity_data.returning_user_access_token('f6cece2afd2ca1d77dc8f397b50cfc2ffe759849') # getting access token
hr_data = get_user_activity_data.get_heart_rate_activity_data(8425197756, access_token)
hr_data2 = get_user_activity_data.get_heart_rate_activity_data(8438327627, access_token)
hr_data['activity_type'] = 'Run'
hr_data2['activity_type'] = 'Ride'
#lap_data = get_user_activity_data.get_activity_laps(8425197756, access_token)
#print(hr_data)
#print(lap_data)

twoact = pd.concat([hr_data, hr_data2])



# Data Manipulation

In [67]:
# Heart Rate Zones and their labels
bins = [0, 150, 160, 205]
labels = ['Zone 1', 'Zone 2', 'Zone 3']

test_df = twoact.explode('data')

hr = test_df.loc[test_df['type'] == 'heartrate']
hr['zone'] = pd.cut(hr['data'], bins=bins, labels=labels)

def get_binned(exploded_hr_data, activity_type=None):
    if activity_type: 
        df = exploded_hr_data.loc[exploded_hr_data['activity_type'] == activity_type]
        counts = pd.cut(df['data'], bins=bins, labels=labels).value_counts().sort_index()
        binned = pd.DataFrame({'zones':counts.index, 'counts':counts}).reset_index(drop=True)
        return binned
    else: 
        counts = pd.cut(exploded_hr_data['data'], bins=bins, labels=labels).value_counts().sort_index()
        binned = pd.DataFrame({'zones':counts.index, 'counts':counts}).reset_index(drop=True)
        return binned
        
binned = get_binned(hr)
fig3 = px.pie(data_frame=binned, values='counts', names='zones', title='Heart Rate Zone Data')

update_menus = []
buttons = [
    {
        'method':'restyle',
        'label':'All',
        'args':[{'values': [get_binned(hr)['counts']]},]
    }
]
for activity_type in hr['activity_type'].unique():
    b = {
            'method':'restyle',
            'label':activity_type,
            'args':[{'values': [get_binned(hr, activity_type)['counts']]},]
        }
    buttons.append(b)
update_menus.append({'buttons':buttons})

fig3.update_layout(updatemenus=update_menus)
fig3.show()

#count = pd.cut(hr['data'], bins=bins, labels=labels) #.value_counts().sort_index()
#binned_counts = pd.DataFrame({'zones':count.index, 'counts':count}).reset_index(drop=True)

/var/folders/22/zrp1n87s1vj__b6sbj99k9240000gn/T/ipykernel_3875/3295768897.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Heart Rate Zones and their labels
bins = [0, 150, 160, 205]
#bins_5 = [0,132,165,181,197,210]
labels = ['Zone 1', 'Zone 2', 'Zone 3']
#labels_5 = ['z1', 'z2', 'z3', 'z4', 'z5']

# Store series data in a useful way
series_data = pd.DataFrame(data={'hr_series': hr_data['data'][0], 'time_series': hr_data['data'][1]})
series_data['zone'] = pd.cut(series_data['hr_series'], bins=bins, labels=labels) # assign each HR value a zone

# Bin data based on heart rate zones
count = pd.cut(series_data['hr_series'], bins=bins, labels=labels).value_counts().sort_index()
# Return a dataframe of binned data
s = pd.DataFrame({'zones':count.index, 'counts':count}).reset_index(drop=True)

print(series_data['zone'])

# Plots

In [ ]:
# Heart Rate Data Plot
fig = px.line(series_data, x="time_series", y="hr_series", title='Heart Rate Data', color_discrete_sequence = ['red'])

# Add Laps to heart rate graph
for i, (name, begin, end, time) in enumerate(zip(lap_data['name'], lap_data['start_index'], lap_data['end_index'], lap_data['moving_time'])):
    if i % 2 == 0:
        color = 'blue'
    else:
        color = 'green'
    fig.add_vrect(x0=begin, x1=end, line_width=0, fillcolor=color, opacity=0.2, annotation_text=name)

# Heart Rate Zones Graphs (pie chart, histogram)
fig2 = px.pie(hr, values='data', labels='index', title='Heart Rate Zone Data')
fig3 = px.histogram(s, x="zones", y="counts", hover_data=s.columns, title='Zone Distribution')

# Activity Lap Data Table
tbl = go.Figure(data=[go.Table(
    header=dict(values=['name','moving time', 'elevation gain', 'average speed', 'average heartrate', 'max heartrate'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[lap_data.name, lap_data.moving_time, lap_data.total_elevation_gain, lap_data.average_speed, lap_data.average_heartrate, lap_data.max_heartrate],
               fill_color='lavender',
               align='left'))
])

tbl.show()
fig.show()
fig2.show()
fig3.show()
